In [ ]:
import requests
import time
import web3
from web3 import Web3, HTTPProvider

"""
Recursively borrow / supply same asset to farm positive NIM
"""

In [ ]:
ALCHEMY_ENDPOINT = "https://rpcapi.fantom.network"
MY_ADDRESS = ""


w3 = Web3(Web3.HTTPProvider(ALCHEMY_ENDPOINT))
w3.isConnected()

## Contract
contract = w3.eth.contract(
    address= Web3.toChecksumAddress("<CONTRACT_ADDRESS>"), 
    abi=CONTRACT_ABI, # ** Contract ABI is of Proxy contract, with .deposit() and .borrow() funcs
)

## Expected Pnl

In [ ]:
initial_capital = 10
capital = initial_capital
total_capital = 0
cRatio = .75 # BTC=70%, DAI=75%
liqBuffer = .05
iterations = 5
carry_pnl = 0


# use GEIST DAI rates
supply = .004
borrow = (.0647 - .0164)
nim = supply + borrow

for n in range(iterations):
    capital *= cRatio - liqBuffer
    carry_pnl += capital * nim
    total_capital += capital
    
roc = carry_pnl / initial_capital * 100.
roc, total_capital

## Deployment

In [ ]:
def waitForBlockConfirmation(func):
    def wrapper(*args, **kwargs):
        emitted = func(*args, **kwargs)
        while True:
            try:
                w3.eth.get_transaction_receipt(emitted)
                break
            except web3.exceptions.TransactionNotFound:
                print('pending transaction')
            time.sleep(5)
        print('block confirmed')
    return wrapper
 
    
@waitForBlockConfirmation
def deposit(contract, amount, token):
    """
    amount in native, ie for DAI, 1 = 1 DAI
    token name, ie DAI
    """
    # Input Data
    depositCall = contract.functions.deposit(
        Web3.toChecksumAddress(TOKENS[token]), # asset
        Web3.toWei(amount, unit="ether"),      # amount
        Web3.toChecksumAddress(MY_ADDRESS),    # from_address
        0,                                     # referral code
    )

    # Transaction call
    # gasLimit ("gas") * gasPrice = max spendable gas (units = eth)
    gasAmount = 2000000
    tx = depositCall.buildTransaction({
        'from': Web3.toChecksumAddress(MY_ADDRESS),
        'nonce': w3.eth.getTransactionCount(MY_ADDRESS),
        'gas': gasAmount,
        'gasPrice': w3.toWei('200', 'gwei'),
        'chainId': 250,
    })
    signed_tx = w3.eth.account.signTransaction(tx, PK)
    emitted = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
    return emitted
    
    
def checkAccount(contract):
    """
    call getUserAccountData() to get available borrow
        totalCollateralETH            uint256
        totalDebtETH                  uint256
        availableBorrowsETH           uint256
        currentLiquidationThreshold   uint256
        ltv                           uint256
        healthFactor                  uint256
    """
    accountData = contract.functions.getUserAccountData(Web3.toChecksumAddress(MY_ADDRESS)).call()
    # parse accountData
    totalCollateral = float(Web3.fromWei(accountData[0], unit="ether"))
    totalDebt = float(Web3.fromWei(accountData[1], unit="ether"))
    availableBorrows = float(Web3.fromWei(accountData[2], unit="ether"))
    liquidationLtv = accountData[3] / 10000. # 8000 = 80% 
    maxLtv = accountData[4] / 10000. # 7500 = 75%
    healthFactor = float(Web3.fromWei(accountData[5], unit="ether"))    

    return {
        "totalCollateral": totalCollateral,
        "totalDebt": totalDebt,
        "availableBorrows": availableBorrows,
        "liquidationLtv": liquidationLtv,
        "maxLtv": maxLtv,
        "healthFactor": healthFactor,
    }

    
@waitForBlockConfirmation
def borrow(contract, amount, token):
    # Input Data
    borrowCall = contract.functions.borrow(
        Web3.toChecksumAddress(TOKENS[token]),  # asset
        Web3.toWei(amount, unit="ether"), # amount
        2,                                      # interestRateMode (where to find this?)
        0,                                      # referral code
        Web3.toChecksumAddress(MY_ADDRESS),     # from_address
    )

    # Transaction call
    # gasLimit ("gas") * gasPrice = max spendable gas (units = eth)
    gasAmount = 2000000
    tx = borrowCall.buildTransaction({
        'from': Web3.toChecksumAddress(MY_ADDRESS),
        'nonce': w3.eth.getTransactionCount(MY_ADDRESS),
        'gas': gasAmount,
        'gasPrice': w3.toWei('200', 'gwei'),
        'chainId': 250,
    })
    signed_tx = w3.eth.account.signTransaction(tx, PK)
    emitted = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
    return emitted


@waitForBlockConfirmation
def repay(contract, amount, token):
    # Input Data
    repayCall = contract.functions.repay(
        Web3.toChecksumAddress(TOKENS[token]),  # asset
        Web3.toWei(amount, unit="ether"), # amount
        2,                                      # interestRateMode (where to find this?)
        Web3.toChecksumAddress(MY_ADDRESS),     # from_address
    )

    # Transaction call
    # gasLimit ("gas") * gasPrice = max spendable gas (units = eth)
    gasAmount = 2000000
    tx = repayCall.buildTransaction({
        'from': Web3.toChecksumAddress(MY_ADDRESS),
        'nonce': w3.eth.getTransactionCount(MY_ADDRESS),
        'gas': gasAmount,
        'gasPrice': w3.toWei('200', 'gwei'),
        'chainId': 250,
    })
    signed_tx = w3.eth.account.signTransaction(tx, PK)
    emitted = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
    return emitted

In [ ]:
######### DEPLOY POSITION
def depositAndBorrow(contract, amount, token, n, liqBuffer):
    """
    Recursively deposit and borrow same asset for n iterations using liqBuffer as risk limit
    """
        
    if n == 0:
        return 
    
    # deposit
    deposit(contract, amount, token)
    
    # check borrow limits
    availableBorrows = checkAccount(contract)['availableBorrows']
    
    # borrow
    borrowAmount = availableBorrows * (1 - liqBuffer)
    borrow(contract, borrowAmount, token)
     
    # repeat
    return depositAndBorrow(contract=contract, amount=borrowAmount, token=token, n=n-1, liqBuffer=liqBuffer)


In [ ]:
depositAndBorrow(
    contract=contract, 
    amount=10, 
    token="<TOKEN>", 
    n=5, 
    liqBuffer=liqBuffer
)

## Repay logic

In [ ]:
# run on machine to repay debt to avoid liquidation as debt accrues
# 
# 

account = checkAccount(geistLendingPool)
totalDebt = account['totalDebt']
totalCollateral = account['totalCollateral']
liquidationLtv = account['liquidationLtv']

## Monitor current Ltv
currentLtv = totalDebt / totalCollateral
healthFactor = liquidationLtv / currentLtv # liquidated at 1
healthFactorBuffer = .1
healthFactorTarget = 1.2 # get back to this health factor

## Solve for totalDebt
# liquidationLtv / currentLtv = healthFactorTarget
# liquidationLtv / (totalDebt / totalCollateral) = healthFactorTarget
# liquidationLtv = healthFactorTarget * (totalDebt / totalCollateral)
# liquidationLtv / healthFactorTarget =  totalDebt / totalCollateral
# debtTarget = liquidationLtv / healthFactorTarget * totalCollateral
debtTarget = liquidationLtv / healthFactorTarget * totalCollateral
debtPayback = totalDebt - debtTarget

# Call .repay()
if healthFactor <= 1 + healthFactorBuffer:
    # repay(geistLendingPool, debtPayback, token)
    pass